In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [3]:
df= pd.read_json("Automotive_5.json",lines = True)

In [4]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B00002243X,"[4, 4]",5,I needed a set of jumper cables for my new car...,"08 17, 2011",A3F73SC1LY51OO,Alan Montgomery,Work Well - Should Have Bought Longer Ones,1313539200
1,B00002243X,"[1, 1]",4,"These long cables work fine for my truck, but ...","09 4, 2011",A20S66SKYXULG2,alphonse,Okay long cables,1315094400
2,B00002243X,"[0, 0]",5,Can't comment much on these since they have no...,"07 25, 2013",A2I8LFSN2IS5EO,Chris,Looks and feels heavy Duty,1374710400
3,B00002243X,"[19, 19]",5,I absolutley love Amazon!!! For the price of ...,"12 21, 2010",A3GT2EWQSO45ZG,DeusEx,Excellent choice for Jumper Cables!!!,1292889600
4,B00002243X,"[0, 0]",5,I purchased the 12' feet long cable set and th...,"07 4, 2012",A3ESWJPAVRPWB4,E. Hernandez,"Excellent, High Quality Starter Cables",1341360000


In [5]:
df = df[['asin','overall','reviewerID']].copy()

In [6]:
df.head()

,asin,overall,reviewerID
0,B00002243X,5,A3F73SC1LY51OO
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4


In [7]:
from sklearn.model_selection import train_test_split
# Splitting train and test data at the start
df_train, df_test = train_test_split(df,test_size=0.2,stratify = df['reviewerID'])

In [8]:
df_train.head()

,asin,overall,reviewerID
13696,B002OUMVWY,5,A3GL6RCS9PP2Y3
8868,B000UOBIZC,5,A1K3O5HTQ3CY0C
7801,B000M61JFC,5,A1C5WS021EL3WO
2156,B00068XCQU,5,A34GSN4VRDFU55
19521,B00APL77NW,5,A5U5T6EWH90O0


In [9]:
df_train = df_train.sort_index(axis=0)

In [10]:
df_train.head()

,asin,overall,reviewerID
0,B00002243X,5,A3F73SC1LY51OO
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4


In [11]:
df_original_train = df_train.copy()
df_train = df_train.pivot(index = "asin" , columns = "reviewerID" , values = "overall")
df_train = df_train.sort_index(axis=1)

In [12]:
df_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00002243Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008BKX5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
X = df_train.values
Y = df_train.sum(axis = 1)
Z =df_train.count(axis =1).astype('float64')
Y = Y.div(Z,axis=0)
cosine
df_new_train = df_train.sub(Y,axis=0).fillna(0)

In [14]:
df_new_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00002243Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008BKX5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
nbrs_train = NearestNeighbors(n_neighbors= 10 , algorithm='auto',metric ='cosine').fit(df_new_train.values)
distances, indices = nbrs_train.kneighbors(df_new_train,return_distance=True)

In [16]:
distances

array([[0.00000000e+00, 9.34939995e-01, 9.38836776e-01, ...,
        9.48565550e-01, 9.53070121e-01, 9.57408229e-01],
       [2.22044605e-16, 8.15362764e-01, 8.33333333e-01, ...,
        9.10912919e-01, 9.16666667e-01, 9.17939006e-01],
       [1.11022302e-16, 7.86191006e-01, 8.69069266e-01, ...,
        9.90833015e-01, 9.94600508e-01, 1.00000000e+00],
       ...,
       [0.00000000e+00, 6.73401368e-01, 7.76393202e-01, ...,
        9.61651751e-01, 9.62476061e-01, 9.65131598e-01],
       [0.00000000e+00, 7.50506270e-01, 8.39894463e-01, ...,
        9.68801471e-01, 9.84459578e-01, 9.92752862e-01],
       [0.00000000e+00, 8.28272625e-01, 8.61754509e-01, ...,
        9.51621293e-01, 9.53547592e-01, 9.59790644e-01]])

In [17]:
indices

array([[   0,  896,  735, ...,  156, 1054,  125],
       [   1, 1212,  663, ...,  698,  396, 1728],
       [   2, 1228,  404, ...,   74,  859, 1222],
       ...,
       [1832, 1604, 1798, ..., 1501,  398, 1564],
       [1833,  169,  466, ..., 1656,  378,  245],
       [1834, 1604, 1378, ...,  809,  174,  614]])

In [18]:
asin_list = df_new_train.index.values.tolist()
index_list = indices.tolist()

In [19]:
index_list

[[0, 896, 735, 1127, 573, 531, 641, 156, 1054, 125],
 [1, 1212, 663, 1493, 1111, 1089, 177, 698, 396, 1728],
 [2, 1228, 404, 721, 1681, 209, 842, 74, 859, 1222],
 [3, 1358, 1017, 885, 1726, 265, 1802, 1571, 1469, 38],
 [4, 338, 481, 291, 1684, 587, 260, 1290, 125, 655],
 [5, 876, 655, 1365, 1269, 1369, 945, 1095, 347, 1598],
 [6, 984, 412, 1720, 1147, 371, 112, 1834, 197, 1529],
 [7, 778, 1422, 248, 31, 1384, 188, 1681, 714, 958],
 [8, 102, 924, 184, 755, 49, 1544, 1243, 1198, 61],
 [9, 271, 169, 1031, 1106, 1331, 1440, 1724, 1364, 367],
 [10, 920, 853, 1652, 847, 881, 675, 1656, 477, 1219],
 [11, 1202, 79, 1145, 167, 1573, 288, 997, 1516, 362],
 [1225, 1226, 1221, 1222, 1223, 1224, 1219, 1230, 1228, 1218],
 [13, 350, 85, 585, 747, 1790, 752, 1675, 187, 621],
 [14, 933, 728, 613, 148, 1529, 1001, 87, 1776, 1152],
 [15, 373, 1337, 404, 1081, 1568, 1272, 406, 1031, 985],
 [16, 1109, 709, 1337, 902, 17, 1230, 275, 1468, 1012],
 [17, 16, 1162, 1109, 709, 1337, 902, 1205, 1230, 588],
 [1225

In [20]:
for i in range(5):
    string = str(asin_list[i]) + " : "
    for x in index_list[i][1:]:
            string += asin_list[x] + " , "
    print(string)

B00002243X : B0018PSASU , B000P17NXQ , B0027VFZE6 , B000EJS9IM , B000E81VN8 , B000HM579K , B00042JSNY , B001TE0IMG , B0002UEMZ2 , 
B00002243Z : B002SIR91A , B000JFJLQA , B004P91FCO , B0024E6UE0 , B001V8U12M , B00068XCQU , B000MY3ML8 , B000BUQOFO , B009OBW5S6 , 
B00008BKX5 : B002X6VXL4 , B000BXKYYI , B000O80B5M , B008DREON6 , B000760FWU , B0012VWUI6 , B0002MADZ8 , B0013VT5N8 , B002UX1T78 , 
B00008RW9U : B003UT3S6Q , B001MXTOKQ , B00167Y3ZO , B009OBW28E , B0009IQXCC , B00D8DR0AO , B005KDOA9Y , B004HCOE7W , B00029WT08 , 
B00008RW9V : B000B68V90 , B000CMDPLM , B0009U7Y2I , B008FV5R0Y , B000F5JM0O , B0009IK81E , B003BJNT0K , B0002UEMZ2 , B000IZ2CIK , 


In [21]:
item_avg = df_original_train.groupby(['asin']).mean()
user_avg = df_original_train.groupby(['reviewerID']).mean()
mean_avg = df_original_train["overall"].mean()
item_map = df_new_train.index.values

In [23]:
item_avg.head()

,overall
asin,
B00002243X,4.777778
B00002243Z,4.333333
B00008BKX5,4.200000
B00008RW9U,4.466667
B00008RW9V,4.666667


In [24]:
df_asin = df.groupby('asin').count()
df_asin = df_asin.index.values

In [26]:
df_asin

array(['B00002243X', 'B00002243Z', 'B00008BKX5', ..., 'B00JGM7Z5Q',
       'B00JGP8I1S', 'B00KIAQ8VW'], dtype=object)

In [27]:
df_reviewerID = df.groupby('reviewerID').count()
df_reviewerID = df_reviewerID.index.values

In [28]:
df_reviewerID

array(['A00473363TJ8YSZ3YAGG9', 'A0473259F6GQNBD88IYN',
       'A09567722SXKYZTV7OFCJ', ..., 'AZTAK8PFB0NLV', 'AZV2U6GU5QA6C',
       'AZYBZEVISD0YX'], dtype=object)

In [29]:
df_dict = {}
for line in df_original_train.values:
    if line[0] in df_dict:
        df_dict[line[0]].append(line[2])
    else:
        df_dict[line[0]] = [line[2]]

In [31]:
predict_for_this = []
count = 0
for i in range(len(df_reviewerID)):
    for j in range(len(df_asin)):
#         print(df_asin[i])
        if df_asin[j] not  in  df_dict or not df_reviewerID[i] in df_dict[df_asin[j]]:
            predict_for_this.append((df_reviewerID[i],df_asin[j]))

In [35]:
df_dict

{'B00002243X': ['A3F73SC1LY51OO',
  'A20S66SKYXULG2',
  'A2I8LFSN2IS5EO',
  'A3GT2EWQSO45ZG',
  'A3ESWJPAVRPWB4',
  'A1ORODEBRN64C',
  'A2R49ZN3G6FTCQ',
  'A3BI8BKIHESDNQ',
  'A1R089P5AS26UE'],
 'B00002243Z': ['A2TQST4IPVA0EC',
  'A3MQAQT8C6D1I7',
  'A1C6RZG7XQZBUZ',
  'A3GT2EWQSO45ZG',
  'A3I23XW4L5ITOG',
  'A30ASPRK2VP8QC'],
 'B00008BKX5': ['A1KKOA3FP2DMBW',
  'A3L1VJMHFWONCB',
  'AT5F1MKA8C2IZ',
  'A4YV49T8Y0X94',
  'A204K1OFE8OB9Y'],
 'B00008RW9U': ['A36HBGDEFPWD0U',
  'A2FJKITDY5UUHJ',
  'A3LYC4BXRDE5RZ',
  'AEL6CQNQXONBX',
  'A3APEIJOQXE142',
  'A24EQC4LXYPT4E',
  'A2H5KD2A9MUZEX',
  'A1LOENXGC0WRI2',
  'AHC2CUX3AWPHC',
  'A2RO3DC31WU3LY',
  'A1QF996642YT9L',
  'A3NEAETOSXDBOM',
  'A33ASQ3O2MIPNP',
  'A1F9Z42CFF9IAY',
  'A3MH75XEELHGY0'],
 'B00008RW9V': ['AP37T7UPLQGWU',
  'A22QKNGN59XYZG',
  'A30XYS6AQ17DNK',
  'A1IU7S4HCK1XK0',
  'A2YDD9A0U77LYS',
  'A2XXRN2ANQDBKO'],
 'B00009RAX7': ['A3155NWLKXEY1I',
  'A9GPCR9WJQCCJ',
  'A2TT3U4U8NMWEL',
  'A3REK3OFONWB1Q'],
 'B00009W3G7': ['

In [33]:
predict_for_this

[('A00473363TJ8YSZ3YAGG9', 'B00002243X'),
 ('A00473363TJ8YSZ3YAGG9', 'B00002243Z'),
 ('A00473363TJ8YSZ3YAGG9', 'B00008BKX5'),
 ('A00473363TJ8YSZ3YAGG9', 'B00008RW9U'),
 ('A00473363TJ8YSZ3YAGG9', 'B00008RW9V'),
 ('A00473363TJ8YSZ3YAGG9', 'B00009RAX7'),
 ('A00473363TJ8YSZ3YAGG9', 'B00009W3G7'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000ATZD5'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000ATZDE'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AXNMO'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AXPDU'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AXRH5'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AXTUY'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AXU02'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AXY62'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AY3X0'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AY60S'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AY9W6'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AYES4'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000AZ9KS'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000DYVN9'),
 ('A00473363TJ8YSZ3YAGG9', 'B0000UUX34'),
 ('A00473363TJ8YSZ3YAGG9', 'B000182DPG'),
 ('A00473363TJ8YSZ3YAGG9', 'B0001L

In [36]:
final_prediction = {}

for i in range(10000):
#__________________________________________________________________________________________________________
# for i in range(len(predict_for_this)): This loop needs to be run instead of above one. 5352407 records: takes time
#__________________________________________________________________________________________________________
    
    item_asin = predict_for_this[i][1]
#     print(item_asin)
    item_userID = predict_for_this[i][0]
#     print(item_asin,item_userID)
    
    predicted_rating = 0
    if item_asin in item_avg.index:
        bi = item_avg.loc[item_asin]['overall'] - mean_avg
    else:
        bi = 0
    if item_userID in user_avg.index:
        bx = user_avg.loc[item_userID]['overall'] - mean_avg
    else:
        bx = 0
            
    baseline_estimate = mean_avg + bi + bx
    predicted_rating = baseline_estimate
#     print(baseline_estimate)
    
    numerator = 0 
    denomenator = 0
    
    if bi != 0 :
        
        item  = np.where(item_map == item_asin)[0][0]
        item_nbrs = indices[item:item+1]
        dist_nbrs = distances[item:item+1]
        cnt = 0;
        k = 0
        
        for j in item_nbrs:
            temp = df_original_train.loc[(df_original_train['asin'] == item_map[j[0]]) &  (df_original_train['reviewerID'] == item_userID)]
            if(temp.size != 0):
                dist = dist_nbrs[cnt:cnt+1]
                numerator += ((1 - dist[0][k])*(temp['overall'].values[0] - (item_avg.loc[temp['asin']]['overall'] - mean_avg) ))

                denomenator += (1 - dist[0][k])
            cnt = cnt + 1
            k+= 1
        if denomenator:
            predicted_rating += numerator/denomenator 

    if item_userID in final_prediction:
        final_prediction[item_userID].append((item_asin,predicted_rating))
    else:
        final_prediction[(item_userID)] = [(item_asin,predicted_rating)]

#___________________________________________________________
#        Sort final_prediction on basis of rating
#___________________________________________________________

In [37]:
final_prediction

{'A00473363TJ8YSZ3YAGG9': [('B00002243X', 4.56117623912837),
  ('B00002243Z', 4.1167317946839255),
  ('B00008BKX5', 3.9833984613505926),
  ('B00008RW9U', 4.250065128017259),
  ('B00008RW9V', 4.450065128017259),
  ('B00009RAX7', 4.533398461350592),
  ('B00009W3G7', 4.583398461350592),
  ('B0000ATZD5', 4.283398461350592),
  ('B0000ATZDE', 4.283398461350592),
  ('B0000AXNMO', 4.552629230581362),
  ('B0000AXPDU', 3.9833984613505926),
  ('B0000AXRH5', 4.402446080398211),
  ('B0000AXTUY', 4.783398461350592),
  ('B0000AXU02', 4.450065128017259),
  ('B0000AXY62', 4.069112747064878),
  ('B0000AY3X0', 4.450065128017259),
  ('B0000AY60S', 4.354827032779164),
  ('B0000AY9W6', 4.247684175636307),
  ('B0000AYES4', 4.783398461350592),
  ('B0000AZ9KS', 3.5333984613505924),
  ('B0000DYVN9', 4.183398461350593),
  ('B0000UUX34', 4.783398461350592),
  ('B000182DPG', 4.672287350239482),
  ('B0001L0DFA', 4.190805868758),
  ('B00020CATC', 4.283398461350592),
  ('B00020CAUG', 3.9083984613505924),
  ('B00020CA

In [38]:
for x in final_prediction:
    final_prediction[x] = sorted(final_prediction[x],key = lambda x:x[1],reverse= True)[:10]

In [39]:
final_prediction

{'A00473363TJ8YSZ3YAGG9': [('B0000AXTUY', 4.783398461350592),
  ('B0000AYES4', 4.783398461350592),
  ('B0000UUX34', 4.783398461350592),
  ('B000221M94', 4.783398461350592),
  ('B00029WY9Y', 4.783398461350592),
  ('B0002CSQ7A', 4.783398461350592),
  ('B0002JJU1O', 4.783398461350592),
  ('B0002JMV06', 4.783398461350592),
  ('B0002KKTT0', 4.783398461350592),
  ('B0002LBGX2', 4.783398461350592)],
 'A0473259F6GQNBD88IYN': [('B0000AXTUY', 5.088954016906148),
  ('B0000AYES4', 5.088954016906148),
  ('B0000UUX34', 5.088954016906148),
  ('B000221M94', 5.088954016906148),
  ('B00029WY9Y', 5.088954016906148),
  ('B0002CSQ7A', 5.088954016906148),
  ('B0002JJU1O', 5.088954016906148),
  ('B0002JMV06', 5.088954016906148),
  ('B0002KKTT0', 5.088954016906148),
  ('B0002LBGX2', 5.088954016906148)],
 'A09567722SXKYZTV7OFCJ': [('B0000AXTUY', 5.533398461350592),
  ('B0000AYES4', 5.533398461350592),
  ('B0000UUX34', 5.533398461350592),
  ('B000221M94', 5.533398461350592),
  ('B00029WY9Y', 5.533398461350592),